In [1]:
import notebook_functions
reload(notebook_functions)
from notebook_functions import *
import os, pandas as pd, cPickle as pickle
from nolearn.lasagne.visualize import plot_conv_activity, plot_conv_weights, plot_occlusion
from lasagne.layers import get_output, get_output_shape
import theano.tensor as T
import theano, StringIO
from skimage.transform import rotate, warp, AffineTransform
from skimage import filters
from scipy import ndimage, misc
import StringIO
%pylab inline

Couldn't import dot_parser, loading of dot files will not be possible.
Populating the interactive namespace from numpy and matplotlib

Using gpu device 0: GeForce GTX 960 (CNMeM is disabled, CuDNN 4004)

pylab import has clobbered these variables: ['ceil', 'floor']
`%matplotlib` prevents importing * from pylab and numpy


In [2]:
input_volume_shape = (261, 261)

In [3]:
val_dir =  "/media/dylan/Science/Kaggle-Data/distracted_drivers/test/"

In [4]:
def image_gen_from_dir_with_filenames(data_dir, batches, size):
    result = [os.path.join(dp, f) for dp, dn, filenames in os.walk(data_dir) 
                  for f in filenames if os.path.splitext(f)[1] == '.jpg']
    image_files = []
    filenames = []
    for file_path in result:
        image_files.append(read_img_file_PIL(file_path, size=input_volume_shape))
        filenames.append(os.path.basename(file_path))
        if len(image_files) == batches:
            X = np.asarray(image_files, dtype=np.float32)
            X = exposure.rescale_intensity(X, out_range=(-1,1))
            yield X, filenames
            image_files = []
            filenames = []
            
    # end result loop, yield any leftover image files
    X = np.asarray(image_files, dtype=np.float32)
    X = exposure.rescale_intensity(X, out_range=(-1,1))
    yield X, filenames

In [5]:
X, filenames = image_gen_from_dir_with_filenames(val_dir, 20, input_volume_shape).next()

In [6]:
dir_name = 'net.vgg.large.l2.5e3.orthog-norm-maxout8-lr.2.steps-size-261'
validation_file_name = "{}/vloss-{}.txt".format(dir_name, dir_name)
model_file_name = "{}/{}.pickle".format(dir_name, dir_name)
best_weights_file_name = "{}/bw-{}.weights".format(dir_name, dir_name)

In [7]:
with open(model_file_name, 'rb') as reader:
    net = pickle.load(reader)

# Random Augmentation

In [8]:
def random_translate(img):
    shift_random = AffineTransform(translation=(randint(-10, 10), randint(-10, 10)))
    min_value = 0 if min(img.ravel()) > 0 else min(img.ravel())
    return np.float32(warp(img, shift_random, mode='constant', cval=min_value))

def random_rotate(img):
    min_value = 0 if min(img.ravel()) > 0 else min(img.ravel())
    return np.float32(rotate(img, randint(-15, 15), mode='constant', cval=min_value))

def random_zoom(img):
    min_value = 0 if min(img.ravel()) > 0 else min(img.ravel())
    scale_random = AffineTransform(scale=(uniform(0.9, 1.1), uniform(0.9, 1.1)))
    return np.float32(warp(img, scale_random, mode='constant', cval=min_value))

def random_compress(img):
    max_v = np.ceil(img.max())
    min_v = np.floor(img.min())
    nd_im = exposure.rescale_intensity(img, out_range=(0, 1)).squeeze()
    nd_im = np.ndarray.astype(nd_im * 255, np.uint8)
    # nd_im = np.ndarray.astype(img * 255, np.uint8)
    im = Image.fromarray(nd_im)
    buf = StringIO.StringIO()
    im.save(buf, "JPEG", quality=np.random.randint(95, 99))
    buf.seek(0)
    im2 = Image.open(buf)
    x1 = exposure.rescale_intensity(np.ndarray.astype(np.array(im2), np.float32), out_range=(min_v, max_v))
    return x1

def random_sharpening(img):
    blurred_f = ndimage.gaussian_filter(img, 0.5)
    filter_blurred_f = ndimage.gaussian_filter(blurred_f, 1)
    alpha = uniform(0.9, 1.2)
    img = blurred_f + alpha * (blurred_f - filter_blurred_f)
    return exposure.rescale_intensity(img, out_range=(-1 , 1))

def random_gamma_correction(img):
    max_v = np.ceil(img.max())
    min_v = np.floor(img.min())
    img = exposure.rescale_intensity(img, out_range=(0,1))
    img = exposure.adjust_gamma(img, uniform(0.2, 0.8))
    return exposure.rescale_intensity(img, out_range=(-1, 1))

In [9]:
def random_aug(img):
    choice = np.random.randint(0,6)
    # choose from 4 different augmentations!
    if choice == 0:
        return random_translate(img)
    elif choice == 1:
        return random_rotate(img)
    elif choice == 2:
        return random_zoom(img)
    elif choice == 3:
        return random_compress(img)
    elif choice == 4:
        return random_sharpening(img)
    else:
        return random_gamma_correction(img)

    
def random_aug_batch(X, aug_algorithm):
    for i in range(X.shape[0]):
        X[i] = aug_algorithm(X[i])
    return X

# Evaluation Proper

In [10]:
load_best_weights(best_weights_file_name, net)

Loaded parameters to layer 'conv2ddnn1' (shape 32x1x7x7).
Loaded parameters to layer 'batchnorm2' (shape 32).
Loaded parameters to layer 'batchnorm2' (shape 32).
Loaded parameters to layer 'batchnorm2' (shape 32).
Loaded parameters to layer 'batchnorm2' (shape 32).
Loaded parameters to layer 'conv2ddnn5' (shape 64x32x3x3).
Loaded parameters to layer 'batchnorm6' (shape 64).
Loaded parameters to layer 'batchnorm6' (shape 64).
Loaded parameters to layer 'batchnorm6' (shape 64).
Loaded parameters to layer 'batchnorm6' (shape 64).
Loaded parameters to layer 'conv2ddnn8' (shape 64x64x3x3).
Loaded parameters to layer 'batchnorm9' (shape 64).
Loaded parameters to layer 'batchnorm9' (shape 64).
Loaded parameters to layer 'batchnorm9' (shape 64).
Loaded parameters to layer 'batchnorm9' (shape 64).
Loaded parameters to layer 'conv2ddnn12' (shape 128x64x3x3).
Loaded parameters to layer 'batchnorm13' (shape 128).
Loaded parameters to layer 'batchnorm13' (shape 128).
Loaded parameters to layer 'bat

In [134]:
image_gen = image_gen_from_dir_with_filenames(val_dir, 100, input_volume_shape)
threaded_gen = threaded_generator(image_gen, num_cached=100)

# aug_probs = []
processed = 0
for X, file_names in threaded_gen:
    all_probs = []
    augments = []
    for idx in range(4):
        X_aug = random_aug_batch(X, random_aug)
        augments.append(X_aug)
    print np.array(augments).reshape(-1, 100, 261, 261).shape
        
    print processed,
    processed+=1
    
    if processed == 3:
        break
        
#     aug_probs.append(all_probs)

(4, 100, 261, 261)
0 (4, 100, 261, 261)
1 (4, 100, 261, 261)
2


In [18]:
image_gen = image_gen_from_dir_with_filenames(val_dir, 100, input_volume_shape)
threaded_gen = threaded_generator(image_gen, num_cached=100)
all_probs = []
for X, file_names in threaded_gen:
    all_probs.extend(net.predict_proba(X.reshape(-1, 1, input_volume_shape[0], input_volume_shape[1])))
        
total_probs.append(all_probs)

In [44]:
len(aug_probs)

798

In [58]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [84]:
mean_aug_probs = []
for batch in aug_probs:
    for (idx, augments) in enumerate(chunks(batch, 4)):
        print idx,
        mean_aug_probs.append(np.mean(augments, axis=0))   

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46

In [92]:
np.array(mean_aug_probs).shape

(79726, 10)

In [90]:
np.array(orig_probs).shape

(79726, 10)

In [110]:
both_probs = np.mean(np.dstack((orig_probs, mean_aug_probs)), axis=2)
both_probs.shape

(79726, 10)

In [128]:
print both_probs[2544]
print orig_probs[2544]
print mean_aug_probs[2544]

[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
[ 0.91080779  0.00645715  0.00683502  0.00640246  0.00603635  0.00780768
  0.00657787  0.00638526  0.00732227  0.03536809]
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]


In [116]:
result = [f for dp, dn, filenames in os.walk(val_dir) 
              for f in filenames if os.path.splitext(f)[1] == '.jpg']
print len(result)
result[:3]

79726


['img_73540.jpg', 'img_9672.jpg', 'img_49295.jpg']

In [113]:
image_gen = image_gen_from_dir_with_filenames(val_dir, 100, input_volume_shape)
threaded_gen = threaded_generator(image_gen, num_cached=100)

all_file_names = []
processed = 0
for _, file_names in threaded_gen:
    all_file_names.extend(file_names)
print len(all_file_names)

79726


In [117]:
df = pd.DataFrame(both_probs, 
                  index = all_file_names, columns=["c0","c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8", "c9"])
df.index.name = 'img'
df.shape

(79726, 10)

In [118]:
df.to_csv("preds.{}-4-random-evals.csv".format(dir_name))

In [119]:
df[:10]

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
img,,,,,,,,,,
img_73540.jpg,0.003872,0.002522,0.125915,0.002078,0.124474,0.732022,0.002204,0.002212,0.002305,0.002396
img_9672.jpg,0.067404,0.008192,0.505686,0.040134,0.009949,0.011723,0.007587,0.131270,0.132086,0.085969
img_49295.jpg,0.065397,0.004449,0.004577,0.127312,0.619758,0.042930,0.121711,0.004321,0.004830,0.004716
img_91517.jpg,0.123476,0.013952,0.119630,0.076834,0.049078,0.503971,0.014284,0.014045,0.061072,0.023657
img_51925.jpg,0.379383,0.007213,0.245508,0.130508,0.127500,0.010328,0.007460,0.007250,0.007601,0.077248
img_87962.jpg,0.008473,0.006391,0.012529,0.129821,0.190044,0.009937,0.124207,0.501242,0.009526,0.007831
img_30198.jpg,0.016030,0.018074,0.151389,0.447696,0.018155,0.129463,0.014491,0.136165,0.051908,0.016628
img_78210.jpg,0.003951,0.127698,0.004284,0.004028,0.004201,0.004024,0.247404,0.004128,0.595400,0.004882
img_19345.jpg,0.093375,0.008706,0.013013,0.008029,0.626881,0.121844,0.097499,0.008485,0.012304,0.009864
